<div style="text-align: right" align="right"><i>Ivy Zhang, December 2023</i></div>

# Advent of Code 2023

Going to try and solve Advent of Code as efficiently as possible this year to get better at Python!

## Day 0: Prep

Just importing things that I might need to keep this file clean!

In [1]:
%run Advent-Utils.ipynb

## [Day 1](https://adventofcode.com/2023/day/1): Trebuchet?!

### Part 1: Find first and last digit, concatenate and sum all of them together

In [5]:
in1 = parse(1)

numbers1 = [re.findall(r'\d', t) for t in in1]
combined_numbers1 = [int(str(t[0]) + str(t[-1])) for t in numbers1]
ans1 = sum(combined_numbers1)

print(ans1)

54968


### Part 2: You must now also take text digits into consideration

In [6]:
mappings = {
    'one': '1',
    'two': '2',
    'three': '3',
    'four': '4',
    'five': '5',
    'six': '6',
    'seven': '7',
    'eight': '8',
    'nine': '9',
}

def parse_calibration(cal, iter):
    for i in iter:
        if cal[i].isdigit():
            return int(cal[i])
        for dig in mappings:
            if cal[i:i+len(dig)] == dig:
                return mappings[dig]


def parse_calibrations(calibrations):
    combined_nums = []
    for cal in calibrations:
        fd, ld = -1, -1
        fd = parse_calibration(cal, range(len(cal)))
        ld = parse_calibration(cal, reversed(range(len(cal))))
        combined_nums.append([fd, ld])
    return combined_nums

in1 = parse(1)
in1 = parse_calibrations(in1)
combined_numbers1 = [int(str(t[0]) + str(t[-1])) for t in in1]
ans1 = sum(combined_numbers1)
print(ans1)


54094


## [Day 2](https://adventofcode.com/2023/day/2): Cube Conundrum

In [71]:
in2 = parse(2, atoms)

### Part 1

Each "hand" can be played in any order and with number of colors. This means my parsing has to be able to take that into account. Given 12 red cubes, 13 blue cubes, and 14 green cubes, see is the hands that the elf is showing are valid

In [58]:
limits = {
    'red': 12,
    'green': 13,
    'blue': 14
}

def find_cube_possibilities() -> int:
    ans = 0
    for game in in2:
        id = game[1]
        for i in range(2, len(game), 2):
            num, color = game[i], game[i+1]
            if num > limits[color]:
                break
        else:
            ans += id
    return ans

print(find_cube_possibilities())

2600


### Part 2

Instead of checking whether or not a game is possible, figure out what the minimum number of each type of color is.

In [72]:
def find_min_cubes() -> int:
    ans = 0
    for game in in2:
        color_mins = {
            'red': 0,
            'green': 0,
            'blue': 0
        }
        for i in range(2, len(game), 2):
            num, color = game[i], game[i+1]
            color_mins[color] = max(color_mins[color], num)
            
        game_power = 1
        for color in color_mins:
            game_power *= color_mins[color]
        ans += game_power
    return ans

print(find_min_cubes())

86036


### [Day 3](https://adventofcode.com/2023/day/3): Gear Ratios

In [58]:
in3 = list(parse(3))

One idea is that from the symbols, you flood fill outwards going into number tiles, then you can use regex to find the numbers that are "valid".

In [59]:
h, l = len(in3), len(in3[0])

val = [[0 for _ in range(l)] for _ in range(h)]
q = deque()

for i, line in enumerate(in3):
    for j, c in enumerate(line):
        if not c.isdigit() and c != '.':
            for di in range(-1, 2):
                for dj in range(-1, 2):
                    ni, nj = i + di, j + dj
                    if 0 <= ni < h and 0 <= nj < l and in3[ni][nj].isdigit():
                        val[ni][nj] = 1
                        q.append((ni, nj))

while len(q) > 0:
    i, j = q.pop()
    # print('popping')
    for dj in range(-1, 2):
        ni, nj = i, j + dj
        if 0 <= ni < h and 0 <= nj < l:
            # print(f'({ni}, {nj}) -> {in3[ni][nj]}')
            if val[ni][nj] == 0 and in3[ni][nj].isdigit():
                # print('adding new')
                val[ni][nj] = 1
                # print(f'Adding ({ni}, {nj})')
                q.append((ni, nj))

ans = 0

# for i in range(h):
#     for j in range(l):
#         print(val[i][j], end='')
#     print()

for i, line in enumerate(in3):
    nums = [(m.group(0), m.start(0)) for m in re.finditer(r'\d+', line)]
    for num, index in nums:
        if val[i][index] > 0:
            ans += int(num)
            # print(f'Adding {num}')

print(ans)


550934


### Part 2

Now we're looking for gears. Instead of flooding each one, should write a function that takes in an index and a marker and floods it in along with giving you the integer. Then for each symbol, keep track of how many ints you got to see whether or not you have a valid "gear".

In [60]:
h, l = len(in3), len(in3[0])

vis = [[0 for _ in range(l)] for _ in range(h)]

def flood_num(i, j):
    val = in3[i][j]
    lj, rj = j-1, j+1
    vis[i][j] = 1

    while lj >= 0 and in3[i][lj].isdigit():
        val = in3[i][lj] + val
        vis[i][lj] = 1
        lj -= 1

    while rj < l and in3[i][rj].isdigit():
        val = val + in3[i][rj]
        vis[i][rj] = 1
        rj += 1

    return int(val)

ans = 0

for i, line in enumerate(in3):
    for j, c in enumerate(line):
        if c == '*':
            adjacent_nums = []
            for di in range(-1, 2):
                for dj in range(-1, 2):
                    ni, nj = i + di, j + dj
                    if 0 <= ni < h and 0 <= nj < l and in3[ni][nj].isdigit() and not vis[ni][nj]:
                        adjacent_nums.append(flood_num(ni, nj))
            if len(adjacent_nums) == 2:
                ans += adjacent_nums[0] * adjacent_nums[1]

print(ans)


81997870
